In [1]:
import pseudopeople as psp
import pandas as pd

pd.set_option('display.max_columns', None) #show all columns
census = psp.generate_decennial_census() #grab Census data
taxes = psp.generate_taxes_w2_and_1099() #grab tax data

C:\Users\dcjc9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\dask\dataframe\__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
# To find how many matches there are between records about a given simulant,
# we need to multiply the number of records about that simulant in the census by
# the number of records about that simulant in taxes
true_matches = census.groupby("simulant_id").size().multiply(
   taxes.groupby("simulant_id").size(), fill_value=0
).sum().astype(int)
print(f"There are {true_matches:,} true matches to find between these datasets!")

There are 9,034 true matches to find between these datasets!


In [3]:
census.head()

,simulant_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year
0,0_2,0_7,Diana,P,Kofron,25,05/06/1994,5112,145th st,NaN,Anytown,WA,00000,Household,Reference person,Female,White,2020
1,0_3,0_7,Anna,A,Kofron,25,09/29/1994,5112,145th st,NaN,Anytown,WA,00000,Household,Other relative,Female,White,2020
2,0_923,0_8033,Gerald,R,Butler,76,11/03/1943,1130,mallory ln,NaN,Anytown,WA,00000,Household,Reference person,Male,Black,2020
3,0_2641,0_1066,Loretta,T,Carley,61,07/71/1958,NaN,delacorte dr,NaN,Anytown,WA,00000,Household,Reference person,Female,White,2020
4,0_2801,0_1138,Richard,R,Jones,73,03/03/1947,950,caribou lane,NaN,Anytown,WA,00000,Household,Reference person,Male,White,2020


In [6]:
census.drop(columns=["age", "date_of_birth", "housing_type", "relationship_to_reference_person", "sex", "race_ethnicity"], inplace = True)

In [ ]:
taxes = taxes[['simulant_id', 'household_id', 'first_name', 'middle_initial', 'last_name', 'mailing_address_street_number', 
'mailing_address_street_name', 'mailing_address_unit_number', 'mailing_address_city', 'mailing_address_state', 'mailing_address_zipcode']]

taxes.rename(columns={"mailing_address_street_number": "street_number", "mailing_address_street_name": "street_name",
                      "mailing_address_unit_number": "unit_number", "mailing_address_city": "city",
                      "mailing_address_state": "state", "mailing_address_zipcode":"zipcode"}, inplace=True)

In [32]:
# Convert all string columns to lowercase
string_cols = census.select_dtypes(include=["object", "string"]).columns
census[string_cols] = census[string_cols].apply(lambda col: col.str.lower())

string_cols = taxes.select_dtypes(include=["object", "string"]).columns
taxes[string_cols] = taxes[string_cols].apply(lambda col: col.str.lower())

In [34]:
positive_pairs = pd.merge(census, taxes, on="simulant_id", suffixes=("_census", "_tax"))
positive_pairs["match"] = 1

In [35]:
def generate_negative_pairs(census, taxes, num_negatives):
    negatives = []
    for _ in range(num_negatives):
        census_row = census.sample(1).iloc[0]
        taxes_row = taxes.sample(1).iloc[0]
        if census_row["simulant_id"] != taxes_row["simulant_id"]:
            negatives.append({
                "first_name_census": census_row["first_name"],
                "middle_initial_census": census_row["middle_initial"],
                "last_name_census": census_row["last_name"],
                "street_number_census": census_row["street_number"],
                "street_name_census": census_row["street_name"],
                "unit_number_census": census_row["unit_number"],
                "city_census": census_row["city"],
                "state_census": census_row["state"],
                "zipcode_census": census_row["zipcode"],
                "first_name_taxes": taxes_row["first_name"],
                "middle_initial_taxes": taxes_row["middle_initial"],
                "last_name_taxes": taxes_row["last_name"],
                "street_number_taxes": taxes_row["street_number"],
                "street_name_taxes": taxes_row["street_name"],
                "unit_number_taxes": taxes_row["unit_number"],
                "city_taxes": taxes_row["city"],
                "state_taxes": taxes_row["state"],
                "zipcode_taxes": taxes_row["zipcode"],
                "match": 0
            })
    return pd.DataFrame(negatives)

negative_pairs = generate_negative_pairs(census, taxes, len(positive_pairs))

In [36]:
census.head()

,simulant_id,household_id,first_name,middle_initial,last_name,street_number,street_name,unit_number,city,state,zipcode,year
0,0_2,0_7,diana,p,kofron,5112,145th st,NaN,anytown,WA,00000,2020
1,0_3,0_7,anna,a,kofron,5112,145th st,NaN,anytown,WA,00000,2020
2,0_923,0_8033,gerald,r,butler,1130,mallory ln,NaN,anytown,WA,00000,2020
3,0_2641,0_1066,loretta,t,carley,NaN,delacorte dr,NaN,anytown,WA,00000,2020
4,0_2801,0_1138,richard,r,jones,950,caribou lane,NaN,anytown,WA,00000,2020


In [37]:
# Prepare the full dataset
data_pairs = pd.concat([
    positive_pairs,
    negative_pairs
], ignore_index=True)

In [50]:
# Fill NaN for all object (string) columns with an empty string
data_pairs[data_pairs.select_dtypes(include=["object"]).columns] = (
    data_pairs.select_dtypes(include=["object"]).fillna("")
)

In [52]:
from textdistance import jaro_winkler, levenshtein

def calculate_similarity(row):
    row["first_name_similarity"] = jaro_winkler(row["first_name_census"], row["first_name_taxes"])
    row["last_name_similarity"] = jaro_winkler(row["last_name_census"], row["last_name_taxes"])
    row["street_number_similarity"] = levenshtein.normalized_similarity(row["street_number_census"], row["street_number_taxes"])
    row["street_name_similarity"] = levenshtein.normalized_similarity(row["street_name_census"], row["street_name_taxes"])
    row["city_similarity"] = levenshtein.normalized_similarity(row["city_census"], row["city_taxes"])
    row["zipcode_similarity"] = levenshtein.normalized_similarity(row["zipcode_census"], row["zipcode_taxes"])
    return row

# Apply similarity functions
data_pairs = data_pairs.apply(calculate_similarity, axis=1)

# Extract features and labels
# X = data_pairs[["first_name_similarity", "last_name_similarity", "street_number_similarity", "street_name_similarity", "city_similarity", "zipcode_similarity"]].values
# y = data_pairs["match"].values